<a href="https://colab.research.google.com/github/yoshimethod/bioinformatics/blob/main/bioinformatics_pr2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.7 MB/s eta 0:00:00


In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import sklearn

In [3]:
gse_mRNA_exprs_normal_selected=pd.read_csv("/content/drive/MyDrive/GSE36376_normal.csv", index_col=0)
gse_mRNA_exprs_tumor_selected=pd.read_csv("/content/drive/MyDrive/GSE36376_tumor.csv", index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/GSE36376_normal.csv'